In [1]:
import pandas as pd
import sklearn
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")
import numpy as np

In [2]:
import xgboost
cols = train.ix[:, "f31":].columns
train.drop(cols, 1, inplace=True)
test.drop(cols, 1, inplace=True)

In [11]:
data = test
ans = []
for i, row in data.iterrows():
    new_week = row["week"] - row["shift"]
    new_year = row["year"]
    if new_week < 1:
        new_week += 52
        new_year -= 1
    index = (data["year"] == new_year) & (data["week"] == new_week) & (data["item_id"] == row["item_id"])
    if index.any():
        data.loc[index, "y"] = data.loc[i, "f30"] * 1.60998758761

In [12]:
data = pd.concat([train, test])

In [13]:
data = data.dropna()
data.loc[:,"y"] = np.log(data["y"] + 1)


In [14]:
X = data.drop(["y"], axis=1)
y = data["y"]

In [15]:
def smape(a, b):
    a = np.exp(a) - 1
    b = np.exp(b) - 1
    return np.mean(np.abs(a - b) / (np.abs(a) + np.abs(b)))
smape = sklearn.metrics.make_scorer(smape, False)

In [16]:
sklearn.model_selection.cross_val_score(xgboost.XGBRegressor(max_depth=4, n_estimators=100), X, y, scoring=smape)

array([-0.14924961, -0.13467938, -0.14029081])

In [203]:
gs = sklearn.model_selection.RandomizedSearchCV(model, {"max_depth": [3, 4, 5, 6], "n_estimators": [200, 300, 500, 800, 1300]}, smape)

In [153]:
gs.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [2, 3, 4]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True,
       scoring=make_scorer(smape, greater_is_better=False), verbose=0)

In [154]:
gs.best_params_

{'max_depth': 4}

In [160]:
# Final results

In [55]:
model = Regressor()
model.fit(X, y)

In [56]:
X_test = test[data.drop(["y"], axis=1).columns]

In [57]:
preds = model.predict(X_test)

In [58]:
for i, row in test.iterrows():
    index = (data["year"] == row["year"]) & (data["week"] == row["week"]) & (data["item_id"] == row["item_id"])
    if index.any():
        preds[i] = data.loc[index, "y"].iloc[0]
preds = np.exp(preds) - 1

In [59]:
sample_submission['y'] = preds
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)

-0.075937770307064056

array([1, 2, 3])

In [51]:
best_shift = 2
class Regressor(sklearn.base.BaseEstimator, sklearn.base.RegressorMixin):
    def __init__(self):
        self.regressor = xgboost.XGBRegressor(max_depth=4)
        self.general_regressor = xgboost.XGBRegressor(max_depth=4)
        
    def fit(self, x, y):
        index = x["shift"] == best_shift
        self.regressor.fit(x[index], y[index])
        self.general_regressor.fit(x, y)
    
    def predict(self, x):
        y = self.general_regressor.predict(x)
        index = x["shift"] == best_shift
        y[index] = self.regressor.predict(x[index])
        for i, (_, row) in enumerate(x.iterrows()):
            if row["shift"] != best_shift:
                index = (x["week"] == row["week"]) & (x["year"] == row["year"]) & (x["item_id"] == row["item_id"]) & (x["shift"] == best_shift)
                if index.any():
                    y[i] = y[index][0]
        return y

In [54]:
sklearn.model_selection.cross_val_score(Regressor(), X, y, scoring=smape)

array([-0.13970038, -0.1331798 , -0.13936167])

In [87]:
test.tail()

,Num,year,week,shift,item_id,f1,f2,f3,f4,f5,...,f22,f23,f24,f25,f26,f27,f28,f29,f30,y
2011,351524,2015,5,1,20438803,55365.0,50688.0,52215.0,67843.0,35507.0,...,30104.0,28432.0,28593.0,47018.0,35588.0,8045.0,29015.0,31345.0,47699.0,NaN
2012,351525,2015,5,1,20438802,22375.0,20980.0,22429.0,33192.0,17703.0,...,10888.0,11925.0,14553.0,19053.0,16459.0,3696.0,11788.0,12257.0,20090.0,NaN
2013,351526,2015,5,1,20438801,19390.0,16777.0,19557.0,25663.0,16033.0,...,11922.0,11187.0,11494.0,16690.0,13141.0,3827.0,11327.0,10732.0,15709.0,NaN
2014,351527,2015,5,1,20438734,745.0,555.0,611.0,682.0,315.0,...,390.0,400.0,253.0,268.0,208.0,175.0,337.0,195.0,200.0,NaN
2015,351528,2015,5,1,20438733,4753.0,5472.0,6372.0,6573.0,3612.0,...,3467.0,2941.0,3757.0,6767.0,4491.0,515.0,2597.0,3981.0,4489.0,NaN


In [4]:
features = ["f{}".format(i) for i in range(1, 31)]
converted = pd.DataFrame(columns=["id", "week"] + features)

In [5]:
for i, row in data.iterrows():
    week = row["year"]*52 + row["week"] - row["shift"]
    if not ((converted["id"] == row["item_id"]) & (converted["week"] == week)).any():
        new = pd.Series({"id": row["item_id"], "week": week})
        new = new.append(row[features])
        converted.loc[i] = new

In [10]:
converted[converted["id"] == 20442076.0].sort("week")

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,id,week,f1,f2,f3,f4,f5,f6,f7,f8,...,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30
0,20442076.0,104675.0,4915.0,38056.0,40185.0,45733.0,59710.0,39982.0,45846.0,43680.0,...,39423.0,41765.0,52590.0,31452.0,44420.0,41865.0,52705.0,36102.0,44163.0,45239.0
2083,20442076.0,104678.0,45733.0,59710.0,39982.0,45846.0,43680.0,48325.0,42685.0,40605.0,...,31452.0,44420.0,41865.0,52705.0,36102.0,44163.0,45239.0,76670.0,30570.0,21627.0
2787,20442076.0,104679.0,59710.0,39982.0,45846.0,43680.0,48325.0,42685.0,40605.0,44601.0,...,44420.0,41865.0,52705.0,36102.0,44163.0,45239.0,76670.0,30570.0,21627.0,47915.0
3470,20442076.0,104680.0,39982.0,45846.0,43680.0,48325.0,42685.0,40605.0,44601.0,41965.0,...,41865.0,52705.0,36102.0,44163.0,45239.0,76670.0,30570.0,21627.0,47915.0,42100.0
4170,20442076.0,104681.0,45846.0,43680.0,48325.0,42685.0,40605.0,44601.0,41965.0,56221.0,...,52705.0,36102.0,44163.0,45239.0,76670.0,30570.0,21627.0,47915.0,42100.0,41805.0
4870,20442076.0,104682.0,43680.0,48325.0,42685.0,40605.0,44601.0,41965.0,56221.0,34260.0,...,36102.0,44163.0,45239.0,76670.0,30570.0,21627.0,47915.0,42100.0,41805.0,35772.0
5564,20442076.0,104683.0,48325.0,42685.0,40605.0,44601.0,41965.0,56221.0,34260.0,39914.0,...,44163.0,45239.0,76670.0,30570.0,21627.0,47915.0,42100.0,41805.0,35772.0,38262.0
6258,20442076.0,104684.0,42685.0,40605.0,44601.0,41965.0,56221.0,34260.0,39914.0,42322.0,...,45239.0,76670.0,30570.0,21627.0,47915.0,42100.0,41805.0,35772.0,38262.0,39251.0
6954,20442076.0,104685.0,40605.0,44601.0,41965.0,56221.0,34260.0,39914.0,42322.0,48903.0,...,76670.0,30570.0,21627.0,47915.0,42100.0,41805.0,35772.0,38262.0,39251.0,44541.0
7639,20442076.0,104686.0,44601.0,41965.0,56221.0,34260.0,39914.0,42322.0,48903.0,42090.0,...,30570.0,21627.0,47915.0,42100.0,41805.0,35772.0,38262.0,39251.0,44541.0,33392.0


array([3, 2, 1])

In [102]:
new

id      20441789.0
week          52.0
dtype: float64

In [53]:
for f in features:
    data.loc[:,f] = np.log(data[f] + 1)